### Ein minimales Zeichenprogramm

- Zeichen mit der Maus Linien und Kreise auf die Leinwand.  
  Eine Linie wird gezogen vom Punkt, wo auf die Leinwand geklickt wird, zum Punkt, wo die Maustaste wieder losgelassen wird.
  Wird vor dem Loslassen 'Shift' gedr&uuml;ckt, wird ein Kreis gezeichnet.
  Wird 'Esc' gedr&uuml;ckt, passiert beim Loslassen nichts.
- Farbe und Linienbreite w&auml;hlen: Wenn Leinwand im Fokus, dr&uuml;cke
  eine Ziffer (neue Linienbreite) oder einer der Buchstaben 'r','g','b', f&uuml;r 
  'rot', 'g&uuml;n' oder 'blau'.
- Dr&uuml;cke 'c', um das Leinwandgekritzel zu l&ouml;schen.

Wir benutzen das [MultiCanvas-Widget](https://ipycanvas.readthedocs.io/en/latest/basic_usage.html).  

Nachstehender Code implementiert Teile des oben beschriebenen Programms.  
Noch zu erg&auml;nzen ist das W&auml;hlen der Linienbreite und das 
Abbrechen mit `Esc`.

In [1]:
import canvas_helpers # Methode remove_all_callbacks(mcanvas) unregistriert alle Callback 
from ipywidgets import Output
from ipycanvas import MultiCanvas, Canvas

In [2]:
def distance(pt1, pt2):
    '''return the distance between the points pt1=(x1,y1) and pt2=(x2,y2)'''
    return ((pt2[0] - pt1[0])**2 + (pt2[1] - pt1[1])**2)**0.5    

In [3]:
A = (3, 4)
B = (-1, 7)
distance(A, B)

5.0

In [4]:
WIDTH = 300
HEIGHT = 200
HEADER_Y = 20
HEADER_XS = (10, 100, 150, 170)
FONT = '12px serif'  # https://ipycanvas.readthedocs.io/en/latest/drawing_text.html
LINEWIDTH = 4

LAYOUT = {'border': '1px solid black'}

In [ ]:
canvas    = Canvas(width=WIDTH, height=HEIGHT, layout = LAYOUT)
canvas

In [ ]:
canvas.close()
del canvas

In [ ]:
mcanvas    = MultiCanvas(3, width=WIDTH, height=HEIGHT, layout = LAYOUT)
mcanvas

In [ ]:
mcanvas.close()

In [6]:
mcanvas.close()    
for c in mcanvas:
    c.close()
del mcanvas

In [5]:
# error_msgs = Output(layout = LAYOUT)
mcanvas    = MultiCanvas(3, width=WIDTH, height=HEIGHT, layout = LAYOUT)

bg, mg, fg  = mcanvas # Background, Middleground, Foreground, 3 uebereinanderliegende Canvas'

fg.font = FONT
fg.text_baseline = 'middle'
fg.line_width = LINEWIDTH
bg.line_width = LINEWIDTH

# display(mcanvas, error_msgs)
display(mcanvas)

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [ ]:
mcanvas.close()

In [ ]:
error_msgs.close()

In [ ]:
bg.close()

In [ ]:
del error_msgs

In [ ]:
state  = {'pt': (0,0), 'mouse_down': False, 'key_pressed': None}
colors = {'r': 'red', 'g': 'green', 'b':'blue'}

def show_state(fg):
    '''show state on fg (foreground)'''
    pts = [(x, HEADER_Y) for x in  HEADER_XS]
    fg.clear()
    
    fg.fill_text('Linewidth: ', *pts[0])
    fg.stroke_lines(pts[1:3])
    
    text = 'key: {}'.format(state['key_pressed'])
    fg.fill_text(text , *pts[3])

In [ ]:
state

In [ ]:
show_state(fg)
distance((0,0), (3,4))

In [ ]:
show_state(fg)

***
Nachstehend definieren wir die Funktionen, welche Maus-Events behandeln.  
Ihr Output (Fehlermeldungen) wird ins Output-Widget `error_msgs` umgeleitet, 
[siehe](https://ipywidgets.readthedocs.io/en/7.7.1/examples/Output%20Widget.html#Debugging-errors-in-callbacks-with-the-output-widget).
***

In [ ]:
@error_msgs.capture(clear_output=True)
def on_mouse_down(x, y):
    '''update state
       zeichne Punkt (Kreis mit Radius halbe line_width) auf mg
    '''
    state['pt'] = (x, y)
    state['mouse_down'] = True
    mg.fill_circle(x, y, radius = bg.line_width / 2)

@error_msgs.capture(clear_output=True)  
def on_mouse_up(x, y):
    '''update state
       zeichen auf bg (background), je nach gedrueckter Taste
       loesche mg (middleground)
    '''
    state['mouse_down'] = False
    pt_current = (x, y)
    pt_clicked = state['pt']
    
    if state['key_pressed'] == 'Shift':
        r = distance(pt_clicked,  pt_current)
        bg.stroke_circle(*pt_clicked, r)
    elif  state['key_pressed'] == 'Escape':   
        pass
    else:    
        line = [pt_clicked,  pt_current]
        bg.stroke_lines(line)
        
    state['key_pressed'] = None
    show_state(fg)
    mg.clear()
    
@error_msgs.capture(clear_output=True)      
def on_key_down(key, *flags):
    '''update state['key_pressed']
       je nach state und gedrueckter Taste: set stroke_style und line_width von bg und fg
       redraw state on fg (foregrond)
    '''
    if state['mouse_down']:
        state['key_pressed'] = key
        
    if key == 'c':
        bg.clear()
    elif key in colors:
        fg.stroke_style = colors[key]
        bg.stroke_style = colors[key]
    elif key in '123456789':
        fg.line_width = int(key)
        bg.line_width = int(key)
        
    show_state(fg)    

In [ ]:
# entferne alle fuer mcanvas registrierten Callbacks 
canvas_helpers.remove_all_callbacks(mcanvas)

# registriere obige Funktionen als Callbacks 
mcanvas.on_mouse_down(on_mouse_down)
mcanvas.on_mouse_up(on_mouse_up)
mcanvas.on_key_down(on_key_down)